In [1]:
import numpy as np
import math
from   itertools import permutations
from   more_itertools import powerset
from operator import itemgetter
#import pandas as pd
from   ucimlrepo import fetch_ucirepo
from   sklearn.model_selection import train_test_split


In [65]:
# Load and split the data
player_names = ['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar', 'chlorides',
                'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']
#player_names = ['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar']
wine_quality = fetch_ucirepo(id=186)
wine_subset  = wine_quality['data']['original'][wine_quality['data']['original']['color'] == 'white']
X = np.array(wine_subset[player_names])
y = np.array(wine_subset['quality'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:

N = set([i for i in range(X.shape[1])])

In [67]:
# Standardise the training data
X_mu, X_sigma, y_mu, y_sigma = X_train.mean(axis=0), X_train.std(axis=0), y_train.mean(axis=0), y_train.std(axis=0)

X_train = (X_train - X_mu) / X_sigma
y_train = (y_train - y_mu) / y_sigma

X_test = (X_test - X_mu) / X_sigma

$$X^TXw = X^Ty$$ 
$$w=(X^TX)^{-1}X^Ty$$

In [68]:
w = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)), X_train.T), y_train)

$$R^2  = \frac{1}{N}w^TX^TXw$$
$$R^2_i = \frac{1}{N}w_i (X^TXw)$$

In [69]:
R2  = np.matmul(w.T,np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
R2i = np.matmul(np.diag(w),np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
print(R2)
print(R2i) 

0.2843126626414035
[-5.15764443e-03  4.48079867e-02  1.16066061e-04 -3.99818409e-02
  1.29593436e-04  2.77336079e-03  1.86662385e-03  1.27245440e-01
  1.04933860e-02  5.15467153e-03  1.36865019e-01]


$$\hat{y} = Xw$$
$$MSE = \frac{1}{N}\sum(y-\hat{y})^2$$

In [70]:
yh  = np.matmul(X_test, w)
MSE = np.matmul((y_test - yh).T, (y_test - yh)) / len(y_test)

In [71]:
w

array([ 0.04371106, -0.2193717 , -0.00827766,  0.41280525, -0.00065067,
        0.09626305, -0.01148575, -0.42336479,  0.10171807,  0.08312901,
        0.3170512 ])

In [72]:
def predict(X_vector):
    y_standard = np.matmul(X_vector, w)
    y_actual   = (y_standard * y_sigma) + y_mu
    return y_actual



Because the model is Linear we can exploit the fact that $\mathbb{E}[f(x)] = f(\mathbb{E}[x])$ to avoid any sampling. We use the single vector $\mathbb{E}[x]$  as the basis for calculating Shapley values.

In [73]:
E_fx   = np.mean(predict(X_test))
f_Ex   = predict(np.mean(X_test, axis=0))
X_mean = np.mean(X_test, axis=0)
print(E_fx, f_Ex)
print(X_mean)
print(X_test[0])

5.89318843511457 5.89318843511457
[-0.06071995 -0.05394598  0.06096112 -0.05766376  0.00878483  0.06399892
  0.04271784 -0.07157771 -0.03416091  0.00292092  0.02209133]
[-1.02448031  0.10495148  0.645294    0.84638847  0.10396831  1.1937415
  0.26148955 -0.12267742 -0.66123017  0.88240435  0.3729047 ]


In [74]:
def value(X_vector, s, X_mean):
    X_eval = np.array(X_mean)
    X_eval[list(s)] = X_vector[list(s)]
    y_actual = predict(X_eval)
    #print(s, X_eval, y_actual)
    return y_actual - E_fx



In [75]:
def marginal(X_vector, j, S, X_mean):
    return value(X_vector, S.union(j), X_mean) - value(X_vector, S, X_mean)



In [76]:
def gamma(N,S):
    return math.factorial(len(S)) * math.factorial(len(N) - len(S) - 1) / math.factorial(len(N))


In [77]:
def phi(X_vector, j, N, X_mean):
    players = N - j
    return np.sum([gamma(N, S) * marginal(X_vector, j, set(S), X_mean) for S in powerset(players)])


In [80]:
X_vector   = X_test[0]
test = {2}
phiT, i = 0, 0
players    = set(N - test)
for S in powerset(players):
    i += 1
    C = players - set(S)
    phiT = phiT +  value(X_vector, set(S).union(test), X_mean) -  value(X_vector, set(S), X_mean)
    print(S, test, C, value(X_vector, set(S), X_mean), value(X_vector, set(S).union(test), X_mean), value(X_vector, set(S).union(test).union(C), X_mean))

phiT / i


() {2} {0, 1, 3, 4, 5, 6, 7, 8, 9, 10} 0.0 -0.004289366252699267 0.4786408658199379
(0,) {2} {1, 3, 4, 5, 6, 7, 8, 9, 10} -0.03735818518431788 -0.04164755143701715 0.4786408658199379
(1,) {2} {0, 3, 4, 5, 6, 7, 8, 9, 10} -0.030911703944668467 -0.035201070197367734 0.4786408658199379
(3,) {2} {0, 1, 4, 5, 6, 7, 8, 9, 10} 0.33095132890321555 0.3266619626505163 0.4786408658199379
(4,) {2} {0, 1, 3, 5, 6, 7, 8, 9, 10} -5.4922393037060147e-05 -0.004344288645737215 0.4786408658199379
(5,) {2} {0, 1, 3, 4, 6, 7, 8, 9, 10} 0.09644162730988182 0.09215226105718166 0.4786408658199379
(6,) {2} {0, 1, 3, 4, 5, 7, 8, 9, 10} -0.0022283106424874077 -0.006517676895186675 0.4786408658199379
(7,) {2} {0, 1, 3, 4, 5, 6, 8, 9, 10} 0.01918485929034297 0.014895493037643703 0.4786408658199379
(8,) {2} {0, 1, 3, 4, 5, 6, 7, 9, 10} -0.05656385973310396 -0.060853225985804116 0.4786408658199379
(9,) {2} {0, 1, 3, 4, 5, 6, 7, 8, 10} 0.0648344229368023 0.06054505668410304 0.4786408658199379
(10,) {2} {0, 1, 3, 4, 5

np.float64(-0.0042893662526994406)

In [83]:
X_vector   = X_test[0]
phi_T = E_fx
phi_i = [phi(X_vector,{player}, N, X_mean)  for player in N]
phi_i_sorted = sorted(enumerate(phi_i), key=itemgetter(1))
phi_i_sorted.reverse()
for player, p_i in phi_i_sorted:
    print(player_names[player].ljust(20), p_i)
print('Total phi:    ', sum(phi_i))
print('Total Reward: ', E_fx + sum(phi_i) )
print('Expected:     ', E_fx)
print('Predicted:    ', predict(X_vector))
print('Delta:        ', predict(X_vector) - (E_fx + sum(phi_i))  )


residual_sugar       0.33095132890321505
alcohol              0.0986349755300136
free_sulfur_dioxide  0.09644162730988107
sulphates            0.06483442293680194
density              0.019184859290342877
chlorides            -5.492239303781808e-05
total_sulfur_dioxide -0.002228310642487571
citric_acid          -0.004289366252699429
volatile_acidity     -0.030911703944668655
fixed_acidity        -0.03735818518431849
pH                   -0.05656385973310471
Total phi:     0.47864086581993787
Total Reward:  6.3718293009345075
Expected:      5.89318843511457
Predicted:     6.3718293009345075
Delta:         0.0
